In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

from skimage.io import imread, imsave
from skimage.measure import find_contours
from skimage.filters import gaussian
from skimage.segmentation import active_contour
from skimage.morphology import binary_erosion
from skimage.draw import polygon
from skimage.transform import warp_polar
from skimage.transform import warp, EuclideanTransform

from scipy.ndimage import distance_transform_edt
from scipy.optimize import fmin, least_squares
from scipy.signal import savgol_filter
from scipy.io import savemat

In [3]:
folder = '/mnt/ff9e5a34-3696-46e4-8fa8-0171539135be/Tweez scope'
exp_date = '2023_11_28'
path = os.path.join(folder, exp_date)
folder_masks = 'contour_masks'
fname = '2023_11_28_10x_1.0x_pAAA_TiTweez_Pos1.ome.tif'
fname_mask = 'mask_' + fname

path_im = os.path.join(path, fname)
path_masks = os.path.join(path, folder_masks, fname_mask)

In [5]:
im_all = imread(path_im)
im_ph = im_all[:,:,:,3]
im_ph = im_ph.astype(float)

In [6]:
nt,nx,ny = im_ph.shape

In [7]:
im_ph.shape

(216, 1024, 1024)

In [11]:
(nt,) + (nx,ny)

(216, 1024, 1024)

In [ ]:
date = '2023_11_15'
pos = [0,1,3]
rads = 100
524, 480
505,520

510, 510

In [16]:
metadata = {
    'Tweez scope': {
        '2023_11_15': {
            '0': {'cx': 524, 'cy': 480, 'radius': 100},
            '1': {'cx': 505, 'cy': 520, 'radius': 100},
            '3': {'cx': 510, 'cy': 510, 'radius': 100},
            # More positions
        },
        # More dates
    },
    'Ti scope': {
        '2023_11_15': {
            '0': {'cx': 600, 'cy': 600, 'radius': 100},
            '1': {'cx': 600, 'cy': 600, 'radius': 100},
            '2': {'cx': 600, 'cy': 600, 'radius': 100},
            '3': {'cx': 600, 'cy': 600, 'radius': 100},
            '6': {'cx': 625, 'cy': 590, 'radius': 110},
            '7': {'cx': 600, 'cy': 580, 'radius': 100},
            '9': {'cx': 605, 'cy': 590, 'radius': 100},
            '17': {'cx': 610, 'cy': 590, 'radius': 100},
            '19': {'cx': 600, 'cy': 600, 'radius': 100},
            # More positions
        },
        # More dates
    }
}

In [33]:
def get_params_for_date(microscope, date, metadata):
    """Retrieve all parameters for a given microscope and date."""
    try:
        date_data = metadata[microscope][date]
        for pos, data in date_data.items():
            print(f"{microscope} for {date}: Pos{pos}, {data}")
    except KeyError:
        print("Data not found for the given microscope and date.")

In [34]:
# Example usage
get_params_for_date('Tweez scope', '2023_11_15', metadata)

Tweez scope for 2023_11_15: Pos0, {'cx': 524, 'cy': 480, 'radius': 100}
Tweez scope for 2023_11_15: Pos1, {'cx': 505, 'cy': 520, 'radius': 100}
Tweez scope for 2023_11_15: Pos3, {'cx': 510, 'cy': 510, 'radius': 100}


In [ ]:
0	Very good	1	600, 600	100
1	Very good	1	600, 600	100
2	Very good	1	600, 600	100
3	Very good	1	600, 600	100
6	Very good	1	625, 590	110
7	Very good	1	600, 580	100
9	Very good	1	605, 590	100
17	Very good	1	610, 590	100
19	Very good	1	600, 600	100